In [21]:
import telebot
import csv
from datetime import datetime
from io import BytesIO
import pandas as pd
from telebot.types import ReplyKeyboardMarkup, KeyboardButton, InlineKeyboardMarkup, InlineKeyboardButton

with open("credentials_02.txt", "r") as file:
    BOT_TOKEN = file.readline().strip().split('=')[1]
    

    
bot = telebot.TeleBot(BOT_TOKEN)

In [20]:
def main_keyboard():
    markup = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=False)
    
    help_button = KeyboardButton("Help")
    markup.add(help_button)
    
    faq_button = KeyboardButton("Frequently Asked Questions")
    markup.add(faq_button)
    
    report_button = KeyboardButton("Report Problem")
    markup.add(report_button)
    
    suggestion_button = KeyboardButton("Suggestions")
    markup.add(suggestion_button)
    
    return markup

def back_keyboard():
    markup = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=False)

    go_back_button = KeyboardButton("Go back to main menu")
    markup.add(go_back_button)
    
    return markup

def read_issues_csv(title=None):
    issues = []
    with open('issues.csv', mode='r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if title is None or row['title'].lower() == title.lower():
                issues.append(row)
    return issues

@bot.message_handler(commands=['issues'])
def send_issues(message):
    title = message.text.split(" ")[1] if len(message.text.split(" ")) > 1 else None
    issues = read_issues_csv(title)
    formatted_issues = format_issues(issues)
    if formatted_issues:
        bot.reply_to(message, formatted_issues)
    else:
        bot.reply_to(message, "No issues found with the given title.")


@bot.message_handler(commands=['start', 'restart'])
def send_welcome(message):
    bot.send_message(message.chat.id, "Welcome! Type /issues to get a list of issues from the 'issues.csv' file.",
                                reply_markup=main_keyboard())

# @bot.message_handler(func=lambda message: True)
# def bot_operator(message):
#     with open('issues.csv', 'rb') as file:
#         content = BytesIO(file.read())
#     content.name = 'issues.csv'
#     bot.send_document(message.chat.id, content)
def write_filtered_issues_csv(title):
    with open('issues.csv', mode='r') as csvfile:
        reader = csv.DictReader(csvfile)
        fieldnames = reader.fieldnames

        with open('filtered_issues.csv', mode='w', newline='') as filtered_csv:
            writer = csv.DictWriter(filtered_csv, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                if row['title'].lower() == title.lower():
                    writer.writerow(row)

@bot.message_handler(commands=['filter_others'])
def filter_others_and_send(message):
    write_filtered_issues_csv("others")
    with open('filtered_issues.csv', 'rb') as file:
        content = BytesIO(file.read())
    content.name = 'filtered_issues.csv'
    bot.send_document(message.chat.id, content)

bot.polling()

ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)

In [22]:
def write_filtered_issues_csv(title):
    df = pd.read_csv('issues.csv')
    filtered_df = df[df['title'].str.lower() == title.lower()]
    filtered_df.to_csv('filtered_issues.csv', index=False)


In [23]:
write_filtered_issues_csv("others")